# Flask REST API Builder

Using templating and Flask, this project should be able to parse a table describing your REST API and turn it into some stub Flask code for you to fill out.

For example, this could be the API for a basic To-Do app (all fields delimited by tabs).

    blueprint-name: api
    prepend-with: /todo/api/v1
    #Method    URL                       Description
    GET       /tasks                     Retrieve list of tasks
    GET       /tasks/<int:task_id>       Retrieve task number <task_id>
    POST      /tasks                     Create a new task 
    PUT       /tasks/<int:task_id>       Update an existing task 
    DELETE    /tasks/<int:task_id>       Delete an existing task 

In [61]:
from flask import Flask, Blueprint
import jinja2
import re
from collections import namedtuple

In [115]:
class Function:
    FUNCTION_TEMPLATE = jinja2.Template(
'''
@{{ blueprint }}.route('{{ url }}', methods=['{{ method|default('GET') }}'])
def {{ name }}({{ args_list|join(', ') }}):
    """
    {{ docstring }}
    """
    # TODO: Complete me!
    raise NotImplemented''')
    
    def __init__(self, url, method, description, blueprint='api', name=None, template=None):
        self.url = url
        self.method = method
        self.description = description
        self.blueprint = blueprint
        self.args = self.get_args()
        self.name = name or self.generate_name()
        self.template = template or self.FUNCTION_TEMPLATE
    
    def get_args(self):
        """
        Find all the arguments in the url. These are usually bits 
        that look like "<int:task_id>" and so on...
        """
        res = re.findall(r'<\w+:(.+)>', self.url)
        return res
    
    def generate_name(self):
        """
        Try to create a stock name for the function given the information
        provided.

        The idea is to create names like the following:
        * get_tasks
        * get_tasks_by_id
        * delete_task_by_id
        """
        name = []
        name.append(self.method.lower())

        # Get the last "word" in the url that isn't a parameter
        for word in reversed(self.url.split('/')):
            if '<' in word or '>' in word:
                continue
            else:
                name.append(word)
                break

        # If there are any arguments, then add "by_[last arg]"
        if self.args:
            name.append('by')
            name.append(self.args[-1])

        return '_'.join(name)
    
    def render(self):
        func = self.template.render(
                blueprint=self.blueprint,
                url=self.url,
                method=self.method,
                name=self.name,
                args_list=self.args,
                docstring=self.description)
        return func
    
    def __repr__(self):
        return '<{}: url="{}" method="{}">'.format(
                self.__class__.__name__,
                self.url,
                self.method)

In [117]:
spec = """
blueprint-name: api
prepend-with: /todo/api/v1
#Method    URL                       Description                       Name
GET       /tasks                     Retrieve list of tasks            retrieve_all_tasks
GET       /tasks/<int:task_id>       Retrieve task number <task_id>
POST      /tasks                     Create a new task 
PUT       /tasks/<int:task_id>       Update an existing task 
DELETE    /tasks/<int:task_id>       Delete an existing task 
"""


Rule = namedtuple('Rule', ('method', 'url', 'description', 'name'))

def parse_spec(spec):
    config = {}
    config['rules'] = []

    for i, line in enumerate(spec.splitlines()):
        # Remove trailing whitespace
        line = line.strip()

        # Skip comments and empty lines
        if line.startswith('#') or not line:
            continue

        # Split by multiple space sections
        groups = re.split(r'\s\s+', line)

        if len(groups) == 1:
            # It's an option
            left, right = groups[0].split(':')

            # If it fails, throw syntax error
            config[left.strip().lower()] = right.strip()
        elif len(groups) == 3:
            method, url, description = groups
            new_rule = Rule(method, url, description, None)
            config['rules'].append(new_rule)
        elif len(groups) == 4:
            method, url, description, name = groups
            new_rule = Rule(method, url, description, name)        
            config['rules'].append(new_rule)
        else:
            raise SyntaxError('Too many fields on line {}'.format(i+1))

    return config

cfg = parse_spec(spec)
pprint(cfg)

{'blueprint-name': 'api',
 'prepend-with': '/todo/api/v1',
 'rules': [Rule(method='GET', url='/tasks', description='Retrieve list of tasks', name='retrieve_all_tasks'),
           Rule(method='GET', url='/tasks/<int:task_id>', description='Retrieve task number <task_id>', name=None),
           Rule(method='POST', url='/tasks', description='Create a new task', name=None),
           Rule(method='PUT', url='/tasks/<int:task_id>', description='Update an existing task', name=None),
           Rule(method='DELETE', url='/tasks/<int:task_id>', description='Delete an existing task', name=None)]}


In [121]:
class APIGenerator:
    def __init__(self, config, function_template=None):
        self.config = config
        self.blueprint = config.get('blueprint-name', 'api')
        self.rules = self.config['rules']
        self.function_template = function_template
        
    def preamble(self):
        """
        Generate the import statements and the blueprint definition.
        """
        template = jinja2.Template("""
from flask import Blueprint

{{ blueprint }} = Blueprint({{ bp_args|join(', ') }})
""")
        bp_args = []
        bp_args.append(repr(self.blueprint))
        bp_args.append('__name__')
        
        if 'prepend-with' in self.config:
            bp_args.append('url_prefix="{}"'.format(self.config['prepend-with']))
            
        rendered = template.render(
                blueprint=self.blueprint,
                bp_args=bp_args)
        return rendered
    
    def functions(self):
        funcs = []
        for rule in self.rules:
            f = Function(
                    rule.url, 
                    rule.method.upper(), 
                    rule.description, 
                    blueprint=self.blueprint, 
                    name=rule.name, 
                    template=self.function_template)
            funcs.append(f)
        return funcs
    
    def render(self):
        lines = []
        
        # Add the preamble
        lines.append(self.preamble())
        
        # Give it a bit of space
        lines.append('')
        lines.append('')
        
        lines.extend(f.render()+'\n' for f in self.functions())
        
        return '\n'.join(lines).strip()

In [122]:
a = APIGenerator(cfg)
rendered = a.render()

with open('api.py', 'w') as f:
    f.write(rendered)